# A quick overview of some of DESDEO's main capabilities

**Note:** This notebook contains the code shown in the three use cases considered in the article titled _DESDEO: an open framework for interactive multiobjective optimization_. Cells containing code found in the article will be marked accordingly (by a comment on the first line). The code shown may vary from the one in the article due to new additions and/or changes in the DESDEO framework. This is to ensure that the code shown can be run and experimented with in the future as well.

This overview should give the novice user interested in DESDEO a brief overview of how the framework may be utilized. We will consider a multiobjective optimization problem with five objectives and two variables. The problem will be treated both as an analytical problem and a data-based problem. We will show how the problem can be solved utilizing MCDM and EMO methods both separately and in tandem (hybridizing them). **Note:** Link to article with additional information to be added later.

## The probelm

The considered problem - the river pollution problem - in the use cases is presented [here](https://ieeexplore.ieee.org/document/35354). Its analytical definiton can be stated as:

\begin{equation}
\begin{array}{rll}
\text{min}  & f_1({\mathbf{x}}) =& - 4.07 - 2.27 x_1 \\ 
\text{min}  & f_2({\mathbf{x}}) =& - 2.60 - 0.03 x_1  - 0.02 x_2 \\
&&\quad - \frac{0.01}{1.39 - x_1^2} - \frac{0.30}{1.39 - x_2^2} \\ 
\text{min}  & f_3({\mathbf{x}}) =& - 8.21 + \frac{0.71}{1.09 - x_1^2} \\ 
\text{min}  & f_4({\mathbf{x}}) =& - 0.96 + \frac{0.96}{1.09 - x_2^2} \\ 
\text{min}  & f_5({\mathbf{x}}) =& \max\{|x_1 - 0.65|, |x_2 - 0.65|\} \\ 
&&\\
\text{s.t.}  && 0.3 \leq x_1, x_2 \leq 1.0, \\
\end{array}
\end{equation}

where each objective is to be minimized subject to box-constraints imposed on the variables.

## Use case 1: solving problems with analytical formulations

If we wish to solve a problem with an analytical formulation, such as the river pollution problem defined below, we can proceed as follows. First, we import some required modules from _desdeo-problem_. As the name suggests, the pacakge _desdeo-problem_ contains modules to that are used to define multiobjective optimization problems. We import the following modules:

In [1]:
# Source code 1
import numpy as np

from desdeo_problem.problem import MOProblem
from desdeo_problem.problem import Variable
from desdeo_problem.problem import ScalarObjective

Next ,we define the objective functions of the river pollution problem first as Python functions and then wrap them inside instances of the `_ScalarObjective` class. The _scalar_ in the name implies that the objective is itself a scalar-valued function (i.e., $\mathbb{R}^n \to \mathbb{R}$ for some positive $n$). The resulting objects are then stored in a list. In code: 

In [2]:
# Source code 2
def f_1(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)  # This step is to guarantee that the function works when called with a single decision variable vector as well.
    return -4.07 - 2.27*x[:, 0]

def f_2(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    return -2.60 - 0.03*x[:, 0] - 0.02*x[:, 1] - 0.01 / (1.39 - x[:, 0]**2) - 0.30 / (1.39 + x[:, 1]**2)

def f_3(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    return -8.21 + 0.71 / (1.09 - x[:, 0]**2)

def f_4(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    return -0.96 - 0.96 / (1.09 - x[:, 1]**2)

def f_5(x: np.ndarray) -> np.ndarray:
    return np.max([np.abs(x[:, 0] - 0.65), np.abs(x[:, 1] - 0.65)], axis=0)

objective_1 = ScalarObjective(name="f_1", evaluator=f_1)
objective_2 = ScalarObjective(name="f_2", evaluator=f_2)
objective_3 = ScalarObjective(name="f_3", evaluator=f_3)
objective_4 = ScalarObjective(name="f_4", evaluator=f_4)
objective_5 = ScalarObjective(name="f_5", evaluator=f_5)

objectives = [objective_1, objective_2, objective_3, objective_4, objective_5]

When defining objectives, it is expected that they may be called with either a single set of decision variables or a set of such sets. I.e.,:

In [3]:
# single set of decision variables
x_single = np.array([0.8, 0.5])

# set of sets
x_multi = np.array([[0.8, 0.5], [0.31, 0.88], [0.34, 0.33]])

# Both sets work when evaluated with the defined objective functions.
# Notice how we get one value for each provided set of decision variables.
print(f"f_1({x_single}) = ", f_1(x_single))
print(f"f_1({x_multi}) = ", f_1(x_multi))

f_1([0.8 0.5]) =  [-5.886]
f_1([[0.8  0.5 ]
 [0.31 0.88]
 [0.34 0.33]]) =  [-5.886  -4.7737 -4.8418]


The variables of the problem are defined in a very similar fashion:

In [4]:
# Source code 3
x_1 = Variable("x_1", 0.5, 0.3, 1.0)
x_2 = Variable("x_2", 0.5, 0.3, 1.0)

variables = [x_1, x_2]

Notice the arguments given to the initializer of 'Variable': the last and second-to-last define the variabels upper and lower bounds, respectively, while the second argument defines a variable's initial value and current value, which is sometimes useful information. The upper and lower bounds are optional; if not provided, appropiate infinims will be assumed for the bounds. The documentation should confirm these claims regarding the arguments:

In [5]:
help(Variable)

Help on class Variable in module desdeo_problem.problem.Variable:

class Variable(builtins.object)
 |  Variable(name: str, initial_value: float, lower_bound: float = -inf, upper_bound: float = inf) -> None
 |  
 |  Simple variable with a name, initial value and bounds.
 |  
 |  Args:
 |      name (str): Name of the variable
 |      initial_value (float): The initial value of the variable.
 |      lower_bound (float, optional): Lower bound of the variable. Defaults
 |          to negative infinity.
 |      upper_bound (float, optional): Upper bound of the variable. Defaults
 |          to positive infinity.
 |  
 |  Attributes:
 |      name (str): Name of the variable.
 |      initial_value (float): Initial value of the variable.
 |      lower_bound (float): Lower bound of the variable.
 |      upper_bound (float): Upper bound of the variable.
 |      current_value (float): The current value the variable holds.
 |  
 |  Raises:
 |      VariableError: Bounds are incorrect.
 |  
 |  Metho

We are now ready to define the multiobjective optimization problem itself. Most of the work has already been done in defining the objectives and variables:

In [6]:
# Source code 4
mo_problem = MOProblem(variables=variables, objectives=objectives)

At this point, we could start solving the problem with various methods found in DESDEO or we can use the problem in various other ways as well to gain additional insight. Since we will be using Synchronous NIMBUS to solve the problem in this use case, we need knowledge of the upper and lower bounds of the possible objective values contained in the set of feasible solutions. For this, we can compute the ideal and nadir points of the problem. The simplest way to compute the ideal and approximate the nadir is to use a pay-off table. A method based on the pay-off table to compute the ideal and (approximation of) the nadir point is found in the _utilities_ module of the _desdeo-mcdm_ package. The ideal and nadir points are then stored as attributes of the `mo_problem` object to have ready access them later on. We proceed as follows:

In [7]:
# Source code 5
from desdeo_mcdm.utilities import payoff_table_method

ideal, nadir = payoff_table_method(mo_problem)

mo_problem.ideal = ideal
mo_problem.nadir = nadir

The ideal and nadir points should give us an idea of the ranges of the objectives. Indeed, this is the case:

In [8]:
print(f"Ideal point: {mo_problem.ideal}")
print(f"Nadir point: {mo_problem.nadir}")
print(f"Ideal strictly better than nadir?: {np.all(mo_problem.ideal <= mo_problem.nadir)}")

Ideal point: [ -6.33999773  -2.8643435   -7.49999957 -11.62612808   0.        ]
Nadir point: [-4.75100227 -2.78676892 -0.32128642 -1.92000058  0.349999  ]
Ideal strictly better than nadir?: True


We can now actually begin solving the river pollution problem using the Synchronous NIMBUS method. As we have already defined an instance of `MOProblem`, initializing and starting the method is straight forward:

In [9]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Source code 6
from desdeo_mcdm.interactive.NIMBUS import NIMBUS

nimbus = NIMBUS(mo_problem)

classification_request, _ = nimbus.start()

The `start` method returns two _requests_ of which the first one is of interest here. Its `contents` attribute is a `dict` the keys of which we can inspect:

In [10]:
classification_request.content.keys()

dict_keys(['message', 'objective_values', 'classifications', 'levels', 'number_of_solutions'])

In the `content` of each _request_, at least the `message` entry exists. This entry contains information on how to proceed:

In [11]:
print(classification_request.content["message"])

Please classify each of the objective values in one of the following categories:
	1. values should improve '<'
	2. values should improve until some desired aspiration level is reached '<='
	3. values with an acceptable level '='
	4. values which may be impaired until some upper bound is reached '>='
	5. values which are free to change '0'
Provide the aspiration levels and upper bounds as a vector. For categories 1, 3, and 5,the value in the vector at the objective's position is ignored. Supply also the number of maximumsolutions to be generated.


We can interact with the method by defining the _response_, an attribute of the _request_, following the instructions contained in the `message` printed above. To help us in this task, we can first inspect the current objective values as:

In [12]:
classification_request.content["objective_values"]

array([ -5.74637016,  -2.77951722,  -6.90637204, -11.62642964,
         0.349999  ])

We may then define our _response_ with the required classifications to continute iterating the method:

In [13]:
# Source code 7
response = {
    "classifications": ["<=", "0", "=", ">=", "<"],
    "levels": [-6.2, 0, 0, -3.0, 0],
    "number_of_solutions": 2,
}

classification_request.response = response

save_request, _ = nimbus.iterate(classification_request)

We got a new request which contains newly computed solutions based on the classifications given. These new solutions are:

In [14]:
save_request.content["objectives"]

[array([-5.74637016, -2.79903209, -6.90637204, -3.        ,  0.13702717]),
 array([-5.54550116e+00, -2.80835322e+00, -7.14632853e+00, -2.39820101e+00,
         5.08847163e-07])]

This request is similar to the earlier one: its `content` attribute will contain at least a `message` entry with instructions on what needs to be defined in the `response` attribute of `save_request`. In Synchronous NIMBUS, the next step would be to indicate whether we would like some of the computed solutions to be saved into an archive for later viewing. Further steps would consist of computing intermediate solutions, providing new classifications, and choosing a new preferred solution. However, we will conclude this use case here. Further information on the implementation of Synchronous NIMBUS in DESDEO can be found [in the documentation of _desde-mcdm_](https://desdeo-mcdm.readthedocs.io/en/latest/notebooks/synchronous_nimbus.html).

## Use case 2: data-based problem

Here we demonstrate how a data-based multiobjective optimization problem may be solved using DESDEO and an evolutionary method found in _desdeo-emo_.

We will assume that the same river pollution problem defined earlier is now computationally _very_ expensive. Thus, we will not try to evaluate the problem itself during optimization. Instead, we have pre-computed a small number (100) of solutions consisting of decision variable and objective values. We can then use these points to train a surrogate model which is computationally less expensive to evaluate and can therefore be readily used in an interactive method. We assume to have already computed the points mentioned earlier and that they are stored in a file `River_Pollution.csv` in a CSV format. The first two columns will consist of the decision variable values and the last five columns will consists of the objective values. We can use this data and together with _desdeo-problem_ and pandas to formulate a data-based problem in DESDEO:

In [15]:
# Source code 8
import pandas as pd
from desdeo_problem.problem import DataProblem


training_data = pd.read_csv("River_pollution.csv")

problem = DataProblem(
    data=training_data, 
    variable_names=["x_0", "x_1"],
    objective_names=["f_1", "f_2", "f_3", "f_4", "f_5"],
    bounds=pd.DataFrame(
        [[0.3, 0.3], [1.0, 1.0]],
        columns=["x_0", "x_1"]),
        index=["lower_bound", "upper_bound"],
)

ModuleNotFoundError: No module named 'desdeo_problem.Problem'